# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

!pip install sklearn-pandas

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np
import nltk
nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_predict
from sklearn import metrics


___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@tainarasm98***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @tainarasm98

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = "Ruffles"

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))
    
    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [101]:
xls_file = pd.ExcelFile('Ruffles.xlsx')

df = xls_file.parse('Treinamento')
df2 = xls_file.parse('Teste')


treinamento = pd.DataFrame({'Treinamento': df['Treinamento']})
classificacao = pd.DataFrame({'Classificação': df['Classificação']})
teste = pd.DataFrame({'Teste': df2['Teste']})
                            
treinamento['Tokenized_treinamento'] = treinamento.apply(lambda row: nltk.word_tokenize(row['Treinamento']), axis=1)


stop = stopwords.words('portuguese')

#treinamento = " ".join(re.findall('[A-Z][^A-Z]*', treinamento)) - importante vooos

regex = re.compile('[%s]' % re.escape(string.punctuation))

tokenized_treinamento_sem_pontuacao = []

for review in treinamento['Tokenized_treinamento']:
    new_review = []
    for token in review: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)

    tokenized_treinamento_sem_pontuacao.append(new_review)
    

tokenized_treinamento_sem_stopwords = []
for frase in tokenized_treinamento_sem_pontuacao:
    new_term_vector = []
    for palavra in frase:
        if not palavra in stopwords.words('portuguese'):
            new_term_vector.append(palavra)
    tokenized_treinamento_sem_stopwords.append(new_term_vector)

    
treinamento['Tokenized_treinamento'] = tokenized_treinamento_sem_stopwords



#"".join(treinamento['Tokenized_treinamento'])

#d = MosesDetokenizer()
#detokens = d.detokenize(tokenized_treinamento_sem_stopwords)
#detokens = "".join(detokens)


vectorizer = CountVectorizer(analyzer='word', tokenizer=lambda doc: doc, lowercase=False)
freq_tweets = vectorizer.fit_transform(treinamento['Tokenized_treinamento'])

#freq_tweets = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False).fit_transform(treinamento['Tokenized_treinamento'])
freq_tweets.toarray()
modelo = MultinomialNB(alpha=1)
modelo.fit(freq_tweets,classificacao['Classificação'])
print(modelo.predict(freq_tweets))


freq_testes = vectorizer.transform(teste['Teste'])
print(modelo.predict(freq_testes))


resultados = cross_val_predict(modelo, freq_tweets, classificacao['Classificação'], cv=10)
print(metrics.accuracy_score(classificacao['Classificação'],resultados))

sentimento=['Relevante','Irrelevante']
print(metrics.classification_report(classificacao['Classificação'],resultados,sentimento))

pd.crosstab(classificacao['Classificação'], resultados, rownames=['Real'], colnames=['Predito'], margins=True)


['Irrelevante' 'Irrelevante' 'Relevante' 'Relevante' 'Irrelevante'
 'Relevante' 'Irrelevante' 'Relevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Relevante' 'Irrelevante' 'Relevante' 'Irrelevante'
 'Relevante' 'Irrelevante' 'Relevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Relevante'
 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevante' 'Relevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante' 'Irrelevante'
 'Irrelevante' 'Irrelevant

Predito,Irrelevante,Relevante,All
Real,,,
Irrelevante,170,77,247
Relevante,20,33,53
All,190,110,300


In [100]:
xls_file = pd.ExcelFile('Ruffles.xlsx')

df = xls_file.parse('Treinamento')
df2 = xls_file.parse('Teste')

treinamento = pd.DataFrame({'Treinamento': df['Treinamento'],
                            'Classificação': df['Classificação']})
                            
treinamento['Tokenized_treinamento'] = treinamento.apply(lambda row: nltk.word_tokenize(row['Treinamento']), axis=1)

relevante = treinamento[(treinamento.Classificação == 'Relevante')]
irrelevante = treinamento[(treinamento.Classificação ==  'Irrelevante')]



all_words = []
all_words2 = []
frequencia = {}
frequencia2 = {}
prob_palavra_relevante = {}
prob_palavra_irrelevante = {}


for percorre_tweets in relevante['Tokenized_treinamento']:
    all_words.extend(percorre_tweets)

for percorre_tweets in irrelevante['Tokenized_treinamento']:
    all_words2.extend(percorre_tweets)

for i in all_words:
    frequencia[i] = all_words.count(i)

for i in all_words2:
    frequencia2[i] = all_words2.count(i)   

for j in frequencia:
    prob_palavra_relevante[j] = ((frequencia[j]+1)/(len(all_words) + (len(frequencia)+len(frequencia2))))

for j in frequencia2:
    prob_palavra_irrelevante[j] = ((frequencia2[j]+1)/(len(all_words) + (len(frequencia)+len(frequencia2))))

prob_rel_irr = df.Classificação.value_counts(True)

prob_teste = df2.Classificação.value_counts(True)*100

prob_teste


Irrelevante    80.5
Relevante      19.5
Name: Classificação, dtype: float64

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [98]:
#### teste = pd.DataFrame({'Teste': df2['Teste']})
                            
teste['Tokenized_teste'] = teste.apply(lambda row: nltk.word_tokenize(row['Teste']), axis=1)

relevante_irrelevante = []

tweets = []


for tweet in teste['Tokenized_teste']:
    tweets.append(tweet)

        
def multiply(numbers):  
    total = 1
    for x in numbers:
        total *= x  
    return total

classificacao_teste = []
diferenca_probabilidades = []

for i in range(len(tweets)):
    sentenca_relevante = []
    sentenca_irrelevante = []
    relevante = []
    irrelevante = []
    for j in tweets[i]:
        if j in frequencia:
            sentenca_relevante.append(((frequencia[j])/(len(all_words)))*prob_rel_irr['Relevante'])
        if j not in frequencia:
            sentenca_relevante.append((1/(len(all_words) + (len(frequencia)+len(frequencia2))))*prob_rel_irr['Relevante'])
            
        if j in frequencia2:
            sentenca_irrelevante.append(((frequencia2[j])/(len(all_words2)))*prob_rel_irr['Irrelevante'])
        if j not in frequencia2:
            sentenca_irrelevante.append((1/(len(all_words2) + (len(frequencia)+len(frequencia2))))*prob_rel_irr['Irrelevante'])
        
    
    relevante.append(multiply(sentenca_relevante))
    irrelevante.append(multiply(sentenca_irrelevante))
    
    for l in range(len(relevante)):
        if relevante[l]>irrelevante[l]:
            classificacao_teste.append('Relevante')
        else:
            classificacao_teste.append('Irrelevante')
            
        
    
verifica_performance = {}
soma = 0
soma_1 = 0
soma_2 = 0
soma_3 = 0
    
for i in range(len(df2.Classificação)):
    if df2.Classificação[i] == classificacao_teste[i]:
        if classificacao_teste[i] == 'Relevante':
            soma+=1
        verifica_performance['Positivos Verdadeiros'] = ((soma/200)*100)
        
        if classificacao_teste[i] == 'Irrelevante':
            soma_1+=1
        verifica_performance['Negativos Verdadeiros'] = (soma_1/200)*100
    else:
        if classificacao_teste[i] == 'Relevante':
            soma_2+=1
        verifica_performance['Positivos Falsos'] = (soma_2/200)*100
        
        if classificacao_teste[i] == 'Irrelevante':
            soma_3+=1
        verifica_performance['Negativos Falsos'] = (soma_3/200)*100
    
           

print(verifica_performance)



"""for i in range(len(classificacao_teste)):
    if classificacao_teste[i]=='Relevante':
        print(tweets[i])"""


{'Positivos Verdadeiros': 7.000000000000001, 'Negativos Verdadeiros': 80.5, 'Positivos Falsos': 0.0, 'Negativos Falsos': 12.5}


"for i in range(len(classificacao_teste)):\n    if classificacao_teste[i]=='Relevante':\n        print(tweets[i])"

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Nosso propósito inicial era classificar os tweets relacionados à marca Lay's (batata), no entanto, após classificar todas as mensagens da base de treinamento e controle, percebemos que não haviam tweets os quais pudéssemos considerar como relevantes, senão os publicados por um mesmo usuário, sendo que o restante se constituía em retweets das postagens deste. Resolvemos, então, classificar as mensagens relativas a outro produto: a batata Ruffles. Isso explica por que o arquivo Excel 'comitado' no Git para o primeiro Check se refere à marca Lay's e não a esta.

Como critério para classificação das mensagens, seguimos o seguinte padrão: todos os tweets que pudessem, de alguma forma, denegrir a imagem da empresa ou servir de base para incrementação e modificação de sua linha de produtos (como as mensagens que insinuam que o produto é para jovens e não para adultos, ou que explicitam nostalgia em relação a algum sabor não mais presente na linha, por exemplo) foram classificados como relevantes, enquanto os demais, como irrelevantes.

Primeiramente, utilizamos o Classificador Naive Bayes de uma das bibliotecas do próprio Python para gerar os resultados: uma forma mais simples e rápida. Eis que este classificou todas as mensagens da nossa base de testes como irrelevantes. Fomos, então, averiguar se isso fazia algum sentido. Na base de testes, classificamos apenas 19,5% das mensagens como relevantes, o que é uma porcentagem pequena, enquanto os outros 80,5% deixamos como irrelevantes. Também percebemos que os tweets da base de testes eram bastante desconexos, isto é, distintos, em comparação à base de treinamento do classificador. Então, por meio de uma operação da mesma biblioteca do Python, dividimos a própria base de treinamento em algumas mensagens para treinar o classificador e outras para testá-lo, de modo que todas as mensagens da base de treinamento foram ora de treino, ora de testes. Feito isso, computamos a acurácia do modelo, isto é, a sua precisão, e obtivemos um valor de, aproximadamente, 68%.

Partimos, então, para a construção do classificador 'na mão', isto é, realizando todos os cálculos por nós mesmas. Obtivemos que 7% dos tweets que eram relevantes verdadeiramente foram classificados como relevantes (positivos verdadeiros), contra 0% que eram irrelevantes, mas foram classificados como relevantes (positivos falsos). Obtivemos também que 80,5% dos irrelevantes foram classificados efetivamente como irrelevantes (negativos verdadeiros), contra 12,5% que eram relevantes, mas foram classificados como irrelevantes (negativos falsos). Percebemos que o classificador tem facilidade em classificar como relevantes mensagens com ampla repetição na base de testes e maior dificuldade com aquelas que não se repetem. Outro fator passível de destaque é que, pelo grande número de mensagens classificadas como irrelevantes na base de treinamento, muitas palavras que constavam nas mensagens relevantes também constavam nas irrelevantes, elevando a chance de o classificador se enganar ao classificar as primeiras na base de testes.

De modo a saber como o classificador trata as mensagens com dupla negação e sarcasmo, analisamos, primeiramente, os resultados dados pelo classificador feito 'na mão' em relação à base de testes. Percebemos, então, que este tem dificuldade em identificar o sarcasmo e a dupla negação, classificando os tweets que contenham esse tipo de conteúdo como irrlevantes. Analisando os resultados obtidos com o classificador em relação à própria base de treinamento (operação supracitada), percebemos que, em alguns momentos, o classificador identificou ironias que possuíam certa relevância (como "tá tão calor que vou abrir um pacote de ruffles pra ver se bate um ventinho") efetivamente como relevantes e também identificou ironias que não possuíam relevância como irrelevantes (por exemplo, "só tenho cara de idiota, dentro dessa moringa com formato de pacote de ruffles tem um cérebro"). Em geral, as frases com dupla negação foram classificadas como irrelevantes (por exemplo, "@emelyalissa não da não, o assassino é uma pessoa não o bicho papão e é uma serie muito fraca, assisti o filme que… https://t.co/npxs3i021t"), embora o exemplo dado tenha sido realmente classificado por nós como irrelevante.

Tendo adquirido uma ampla experiência nesse período de verdadeira imersão em análise de sentimentos para a execução do projeto, garantimos à empresa contratante que continuar financiando o nosso projeto é o melhor investimento que esta pode efetuar. Estamos cheias de insights para aperfeiçoar o classificador ao máximo. O primeiro passo é ampliar a base de treinamento: quanto mais habituado a novas "situações" (tipos de conteúdo), o classificador estiver, mais preciso ele será. O segundo passo é limpar ainda mais o texto da base de treino, reduzindo palavras de mesma família a um mesmo radical (para que todas as ocorrências de um mesmo tipo de palavra sejam classificadas como sendo a mesma palavra), criando uma lista das gírias e das abreviações para Internet mais populares em português para excluí-las da base de treinamento, separando as palavras escritas juntas, isto é, sem espaço (o que é muito comum no meio virtual), reduzindo expressões escritas de forma anormal a expressões regulares ("churrascooooo" - "churrasco) e removendo as URL's dos tweets. Feito isso e incrementando outras ideias que forem surgindo em meio à empreitada, construiremos o melhor classificador já visto, capaz de identificar quase que com precisão de 100% ironias, sarcasmo e classificando as mensagens, com êxito, em relevantes e irrelevantes.


É perceptível que não podemos alimentar a base de Treinamento automaticamente utilizando o próprio classificador porque, embora treinemos este com o maior número possível de tweets, ainda assim ele não possuirá uma inteligência humana, isto é, há possibilidade de que surjam tipos de conteúdo, sentimentalismo, com os quais ele não terá familiaridade e classificará de forma errada. Quanto mais classificações erradas ele fizer na base de treinamento, maior a propagação do erro e menor a sua precisão nos testes. A base de treinamento é uma referência. Esta tem de estar de acordo com o que nós, humanos, consideramos como relevante, para que o classificador aprenda a olhar as mensagens com os nossos olhos e erre cada vez menos em suas classificações, nos mostrando apenas aquilo que realmente queremos ver. 

Embora o classificador Naive Bayes seja muito comum em classificação de textos, uso o qual demonstramos nesse projeto, este também pode ser utilizado em diagnóstico médico - para detectar a probabilidade de um diagnóstico estar certo ou errado para determinada doença ou para determinar a probabilidade de um indivíduo continuar a ter uma doença após o tratamento, como câncer, por exemplo-, Engenharia de Controle - para identificar possíveis falhas na movimentação e execução de tarefas de robôs, por exemplo.

Para tornar o classificador cada vez mais preciso, em categorização de textos, ao menos, é possível implementar todas as soluções descritas anteriormente (em nossa defesa para a continuação do financiamento de nosso projeto pela empresa).